# Дообучение модели Gemma-2-2b

* Дообучим предобученную большую языковую модель `Gemma-2-2b` от компании `Google`.
* Модель `Gemma-2-2b` построена на улучшенной архитектуре GPT и имеет 2 млрд параметров. Ссылка на модель: <https://huggingface.co/google/gemma-2-2b>
* Для обучения используем библиотеку `transformers` от `Hugging Face` и `PyTorch` в качестве backend.
* Ссылка на данные: <https://github.com/yandex/geo-reviews-dataset-2023>

## Импорты

In [1]:
import os
import re
import random
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
from datasets import Dataset
from dotenv import load_dotenv
from huggingface_hub import login
from peft import LoraConfig, get_peft_model
from transformers import (AutoModelForCausalLM, AutoTokenizer,
                          DataCollatorForLanguageModeling, Trainer,
                          TrainingArguments)

## Настройки

In [2]:
def seed_all(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    random.seed(seed)

# Для повторяемости результатов
SEED = 42
seed_all(SEED)
# Загрузка переменных из .env файла
load_dotenv()
# Чтение токена
token = os.getenv("HUGGING_FACE_ACCESS_TOKEN")
# Авторизация в Hugging Face
login(token)
# Отключим вывод предупреждений
warnings.filterwarnings('ignore')
# Устройство для тензорных вычислений и хранения модели в памяти.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## Загрузка модели

* Используем `FP16` - половинную точность (half-precision) для весов модели. Это позволяет экономить память GPU без существенного снижения качества обучения модели.

In [ ]:
model_name = "google/gemma-2-2b"
tokenizer = AutoTokenizer.from_pretrained(model_name, token=token)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # Половинная точность
    token=token,
)
model = model.to(device)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

## Дообучение LoRA

`LoRA` (Low-Rank Adaptation of Large Language Models) - метод, который позволяет значительно сократить сложность и время дообучения больших языковых моделей  (LLM), за счёт заморозки матрицы весов. Вместо неё создаются и обучаются две малые матрицы весов низкой размерности. Итоговый результат получается сложением выходов новых матриц и замороженной матрицы. 

Метод основан на гипотезе, что для задачи дообучения LLM большинство параметров являются избыточными. Говорят, что при дообучении LLM имеет низкий внутренний ранг (intrinsic rank).

Количество обучаемых параметров до применения LoRA.

In [4]:
def count_params(model):
    def pretty_number(num):
        return "{:,}".format(num).replace(",", " ")

    all_params = sum(p.numel() for p in model.parameters())
    grad_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print("Параметров:", pretty_number(all_params))
    print("Обучаемых параметров:", pretty_number(grad_params))

count_params(model)

Параметров: 2 614 341 888
Обучаемых параметров: 2 614 341 888


Применим LoRA к модели.

In [5]:
LORA_RANK = 6

# Настройки LoRA
lora_config = LoraConfig(
    r=LORA_RANK,  # Ранг малых матриц, веса которых мы будем обучать
    lora_alpha=LORA_RANK,  # (alpha / r) - множитель для выходов малых обучаемых матриц
    lora_dropout=0.1,  # Dropout регуляризация для малых обучаемых матриц
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora_config)

Количество обучаемых параметров после применения LoRA.

In [6]:
count_params(model)

Параметров: 2 615 539 968
Обучаемых параметров: 1 198 080


Видим, что общее количество весов модели выросло, как раз на количество обучаемых весов.

Примечание:

* Хотя мы обучаем всего около 1,2 млн параметров с помощью LoRA, скорость обучения будет примерно такой же, как если бы мы обучали модель с 200 млн параметров без LoRA. Это происходит потому, что на этапе прямого хода всё равно требуется использовать все 2,6 млрд параметров базовой модели для вычисления выходов, и только на этапе обратного распространения ошибки начинает работать оптимизация LoRA.

## Подготовка данных

* Очистим данные.
* Возьмём только подвыборку из первых 15000 строк. Это уменьшит скорость обучения в 33 раза и позволит проверить гипотезы, подобрать гиперпараметры, проверить генерацию ответов моделью на известных ей входных данных. Обучение же модели на всех данных может занять сутки и более на GPU уровня T4.

In [7]:
df = pd.read_csv("../data/prepared/data.csv")
df["name_ru"] = df["name_ru"].fillna("")
df["text"] = df["text"].str.replace("\\n", " ")
df = df.drop_duplicates(ignore_index=True)
df_slice = df[:15000]

dataset = Dataset.from_pandas(df_slice)
dataset

Dataset({
    features: ['address', 'name_ru', 'rating', 'rubrics', 'text'],
    num_rows: 15000
})

## Векторизация данных

* Мы добавили в конец входного текста токен \<eos> (End of sequence). Это поможет модели на этапе инференса «понять», когда нужно остановиться и не испортить сгенерированный отзыв, продолжая текст.

In [8]:
MAX_LENGTH = 128  # Максимальная длина текстов после токенизации

def preprocess_function(examples):
    # Формируем входной текст
    inputs = [
        f"Аddress: {address}\nName: {name}\nRating: {rating}\nKeywords: {rubrics}\nReview: {text}{tokenizer.eos_token}"
        for address, name, rating, rubrics, text in zip(
            examples["address"],
            examples["name_ru"],
            examples["rating"],
            examples["rubrics"],
            examples["text"],
        )
    ]
    # Векторизуем входной текст и приводим его к выбранной длине
    return tokenizer(
        inputs, max_length=MAX_LENGTH, truncation=True, padding="max_length"
    )

# Применяем препроцессинг
tokenized_dataset = dataset.map(
    preprocess_function, batched=True, remove_columns=dataset.column_names
)
tokenized_dataset

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 15000
})

## Разделение данных

In [9]:
split_dataset = tokenized_dataset.train_test_split(test_size=0.1, seed=SEED)
split_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 13500
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 1500
    })
})

## Гиперпараметры обучения

In [10]:
BATCH_SIZE = 4
GRAD_ACCUM = 1
N_EPOCHS = 3
LEARNING_RATE = 5e-4
WEIGHT_DECAY = 0.01

n_steps = int(round(N_EPOCHS * len(split_dataset["train"]) / BATCH_SIZE / GRAD_ACCUM, 0))
print("Всего шагов обучения:", n_steps)

Всего шагов обучения: 10125


In [11]:
training_args = TrainingArguments(
    output_dir="../models/gemma-2-2b-lora-finetuned",  # Директория сохранения модели
    overwrite_output_dir=True,  # Перезапись директории при каждом запуске обучения
    save_strategy="epoch",  # Сохраняем модель в конце каждой эпохи
    load_best_model_at_end=True,  # Сохраняем лучшую по метрике модель в конце обучения
    metric_for_best_model="loss",  # Метрика для оценки модели
    save_total_limit=1,  # Сохранить только одну модель
    eval_strategy="epoch",  # Оценивать модель в конце каждой эпохи
    learning_rate=LEARNING_RATE,  # Скорость обучения
    lr_scheduler_type="linear",  # Линейное снижение скорости обучения между шагами
    per_device_train_batch_size=BATCH_SIZE,  # Размер батча данных на этапе обучения
    per_device_eval_batch_size=BATCH_SIZE,  # Размер батча данных на этапе оценивания
    gradient_accumulation_steps=GRAD_ACCUM,  # Накапливать градиенты N шагов и подстроить веса
    num_train_epochs=N_EPOCHS,  # Количество эпох
    weight_decay=WEIGHT_DECAY,  # Коэффициент регуляризации
    fp16=True,  # Использовать формат чисел FP16 (Floating point 16-bit)
    logging_steps=500,  # Через сколько шагов выводить логи
    report_to="none",  # Выводить логи только в стандартный вывод
)

In [12]:
# Объект, который извлекает данные батчами для обучения
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # Отключаем режим маскирования текста MLM (Masked language modeling)
)

In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=split_dataset["train"],
    eval_dataset=split_dataset["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
)

## Обучение модели

In [14]:
trainer.train()

  0%|          | 0/10125 [00:00<?, ?it/s]

{'loss': 2.0711, 'grad_norm': 0.8655328750610352, 'learning_rate': 0.00047530864197530866, 'epoch': 0.15}
{'loss': 1.98, 'grad_norm': 0.8707149624824524, 'learning_rate': 0.00045066666666666665, 'epoch': 0.3}
{'loss': 1.9586, 'grad_norm': 0.7667140364646912, 'learning_rate': 0.0004259753086419753, 'epoch': 0.44}
{'loss': 1.941, 'grad_norm': 0.714769184589386, 'learning_rate': 0.00040128395061728395, 'epoch': 0.59}
{'loss': 1.9204, 'grad_norm': 0.7591819167137146, 'learning_rate': 0.0003765925925925926, 'epoch': 0.74}
{'loss': 1.9158, 'grad_norm': 0.817223072052002, 'learning_rate': 0.00035190123456790124, 'epoch': 0.89}


The 'batch_size' argument of HybridCache is deprecated and will be removed in v4.49. Use the more precisely named 'max_batch_size' argument instead.
The 'batch_size' attribute of HybridCache is deprecated and will be removed in v4.49. Use the more precisely named 'self.max_batch_size' attribute instead.


  0%|          | 0/375 [00:00<?, ?it/s]

{'eval_loss': 1.8898636102676392, 'eval_runtime': 54.9415, 'eval_samples_per_second': 27.302, 'eval_steps_per_second': 6.825, 'epoch': 1.0}
{'loss': 1.8951, 'grad_norm': 0.984058678150177, 'learning_rate': 0.00032725925925925924, 'epoch': 1.04}
{'loss': 1.8463, 'grad_norm': 0.9858825206756592, 'learning_rate': 0.0003025679012345679, 'epoch': 1.19}
{'loss': 1.8379, 'grad_norm': 1.5879651308059692, 'learning_rate': 0.00027792592592592593, 'epoch': 1.33}
{'loss': 1.8382, 'grad_norm': 1.0335925817489624, 'learning_rate': 0.0002532345679012346, 'epoch': 1.48}
{'loss': 1.8281, 'grad_norm': 0.8982611298561096, 'learning_rate': 0.0002285432098765432, 'epoch': 1.63}
{'loss': 1.8406, 'grad_norm': 1.0581979751586914, 'learning_rate': 0.00020385185185185184, 'epoch': 1.78}
{'loss': 1.8204, 'grad_norm': 0.994371235370636, 'learning_rate': 0.0001791604938271605, 'epoch': 1.93}


  0%|          | 0/375 [00:00<?, ?it/s]

{'eval_loss': 1.8506786823272705, 'eval_runtime': 52.6636, 'eval_samples_per_second': 28.483, 'eval_steps_per_second': 7.121, 'epoch': 2.0}
{'loss': 1.7952, 'grad_norm': 0.93693608045578, 'learning_rate': 0.00015446913580246914, 'epoch': 2.07}
{'loss': 1.7648, 'grad_norm': 0.9951069951057434, 'learning_rate': 0.00012977777777777779, 'epoch': 2.22}
{'loss': 1.7472, 'grad_norm': 1.2637635469436646, 'learning_rate': 0.00010508641975308642, 'epoch': 2.37}
{'loss': 1.7589, 'grad_norm': 1.1654475927352905, 'learning_rate': 8.044444444444444e-05, 'epoch': 2.52}
{'loss': 1.7515, 'grad_norm': 0.9364428520202637, 'learning_rate': 5.575308641975309e-05, 'epoch': 2.67}
{'loss': 1.7438, 'grad_norm': 1.1637076139450073, 'learning_rate': 3.106172839506173e-05, 'epoch': 2.81}
{'loss': 1.7527, 'grad_norm': 0.9458349347114563, 'learning_rate': 6.4197530864197525e-06, 'epoch': 2.96}


  0%|          | 0/375 [00:00<?, ?it/s]

{'eval_loss': 1.838716745376587, 'eval_runtime': 51.3445, 'eval_samples_per_second': 29.214, 'eval_steps_per_second': 7.304, 'epoch': 3.0}
{'train_runtime': 3318.883, 'train_samples_per_second': 12.203, 'train_steps_per_second': 3.051, 'train_loss': 1.8494055507330247, 'epoch': 3.0}


TrainOutput(global_step=10125, training_loss=1.8494055507330247, metrics={'train_runtime': 3318.883, 'train_samples_per_second': 12.203, 'train_steps_per_second': 3.051, 'total_flos': 6.3007869468672e+16, 'train_loss': 1.8494055507330247, 'epoch': 3.0})

Выводы:

* Видим, что в конце обучения метрика `eval_loss` (Кросс-энтропия) всё ещё уменьшается. Следовательно, модель ещё не дообучена.
* Можно увеличить количество эпох, или немного увеличить скорость обучения.

## Проверка генерации текста

Создадим запрос (prompt) из данных на которых модель не обучалась.

In [15]:
# Возьмём векторизованный текст из тестового датасета
vectorized_text = split_dataset["test"][2]["input_ids"]
# Декодируем из вектора обратно в текст
text = tokenizer.decode(vectorized_text, skip_special_tokens=True)
# Разделим текст на запрос и эталонный ответ
prompt, text = text.split("Review: ")
prompt = prompt + "Review: "
print(prompt, text)

Аddress: Санкт-Петербург, проспект Художников, 27, корп. 1
Name: КрасКи
Rating: 5
Keywords: Салон красоты
Review:  Впервые побывала в этой студии, делала стрижку. Сразу видно, что студия новая, очень уютная и чистая. Мастер приятная девушка, все сделала быстро и сразу поняла чего я хочу. Обязательно вернусь снова! 



In [16]:
# Параметры генерации текста
generation_config = {
    "max_length": 128,
    "pad_token_id": tokenizer.eos_token_id,
    "eos_token_id": tokenizer.eos_token_id,
    "do_sample": True,
    "num_beams": 1,
    "temperature": 0.95,
    "top_k": 10,
    "top_p": 0.95,
}

# Векторизация текста
inputs = tokenizer(prompt, return_tensors="pt").to(device)
# Генерация текста
outputs = model.generate(**inputs, **generation_config)
response = outputs[0][outputs.shape[-1] :]
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Аddress: Санкт-Петербург, проспект Художников, 27, корп. 1
Name: КрасКи
Rating: 5
Keywords: Салон красоты
Review: 25 лет я хожу в салон КрасКи на Проспекте Художников. Замечательные мастера, очень профессионалы, которые делают всё на высшем уровне. Приятная дружелюбная атмосфера. Спасибо вам большое!



Выводы:

* Сгененрированный текст достаточно качественный. Есть небольшие ошибки в грамматике.
* Сгененрированный текст не совпадает с истинным, но они и не должны совпадать. Мы использовали для генерации запрос, который модель раньше не видела. Поэтому возможно огромное количество генераций текстов, и все из них правильные, с точки зрения модели.
* Если бы мы использовали для генерации уже известный модели запрос, то мы проверяли бы лишь степень переобученности модели, а не её обобщающую способность.